# VAE Training

In [1]:
import torch as t
import torch.nn.functional as F
from tqdm import tqdm
from models.VariationalAutoencoder import VariationalAutoencoder
from torchvision import datasets, transforms

bs = 128
train_ds = datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
train_dl = t.utils.data.DataLoader(dataset=train_ds, batch_size=bs, shuffle=True, drop_last=True)

device = t.device('cuda') if t.cuda.is_available() else 'cpu'
model = VariationalAutoencoder(train_ds[0][0][None], in_c=1, enc_out_c=[32, 64, 64, 64],
                    enc_ks=[3, 3, 3, 3], enc_pads=[1, 1, 0, 1], enc_strides=[1, 2, 2, 1],
                    dec_out_c=[64, 64, 32, 1], dec_ks=[3, 3, 3, 3], dec_strides=[1, 2, 2, 1],
                    dec_pads=[1, 0, 1, 1], dec_op_pads=[0, 1, 1, 0], z_dim=2)
model.cuda(device)
model.train()

VariationalAutoencoder(
  (enc_conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.25)
    )
    (3): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm2d(64, e

In [2]:
def vae_kl_loss(mu, log_var):
    return -.5 * t.sum(1 + log_var - mu ** 2 - log_var.exp())

def vae_loss(y_pred, mu, log_var, y_true):
    r_loss = F.binary_cross_entropy(y_pred, y_true, reduction='sum')
    kl_loss = vae_kl_loss(mu, log_var)
    return r_loss + kl_loss

In [3]:
lr = .0005
for epoch in tqdm(range(20)):
    optimizer = t.optim.Adam(model.parameters(), lr=lr / (epoch * 2 + 1), betas=(.9, .99), weight_decay=1e-2)
    for i, (data, _) in enumerate(train_dl):
        data = data.to(device)
        optimizer.zero_grad()
        pred, mu, log_var = model(data)
        loss = vae_loss(pred, mu, log_var, data)
        loss.backward()
        optimizer.step()
        if i % 33 == 0:
            print(loss)

print(loss)

  0%|          | 0/20 [00:00<?, ?it/s]

tensor(108783.1250, device='cuda:0', grad_fn=<AddBackward0>)
tensor(62251.4727, device='cuda:0', grad_fn=<AddBackward0>)
tensor(45925.7930, device='cuda:0', grad_fn=<AddBackward0>)
tensor(33259.5938, device='cuda:0', grad_fn=<AddBackward0>)
tensor(29036.8750, device='cuda:0', grad_fn=<AddBackward0>)
tensor(28326.6035, device='cuda:0', grad_fn=<AddBackward0>)
tensor(25913.1426, device='cuda:0', grad_fn=<AddBackward0>)
tensor(25414.1348, device='cuda:0', grad_fn=<AddBackward0>)
tensor(25909.0078, device='cuda:0', grad_fn=<AddBackward0>)
tensor(25462.0059, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24686.7383, device='cuda:0', grad_fn=<AddBackward0>)
tensor(23606.9512, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22902.1191, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24357.7031, device='cuda:0', grad_fn=<AddBackward0>)
tensor(23244.5645, device='cuda:0', grad_fn=<AddBackward0>)


  5%|▌         | 1/20 [00:19<06:19, 19.98s/it]

tensor(23037.9980, device='cuda:0', grad_fn=<AddBackward0>)
tensor(23119.1738, device='cuda:0', grad_fn=<AddBackward0>)
tensor(24233.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22439.0488, device='cuda:0', grad_fn=<AddBackward0>)
tensor(23018.7676, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22614.6309, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21595.1387, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22400.3887, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22202.3477, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22966.3867, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22581.9355, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22255.6836, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21529.6387, device='cuda:0', grad_fn=<AddBackward0>)
tensor(22259.2793, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21393.3711, device='cuda:0', grad_fn=<AddBackward0>)


 10%|█         | 2/20 [00:40<06:01, 20.06s/it]

...

tensor(20421.5742, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21626.6543, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20620.6426, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21470.6055, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20645.3281, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20720.9844, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21151.9727, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21443.3496, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21731.2148, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20645.4023, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21662.0234, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20963.5645, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20453.7891, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20917.6094, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20432.3555, device='cuda:0', grad_fn=<AddBackward0>)


 95%|█████████▌| 19/20 [06:22<00:20, 20.22s/it]

tensor(20687.7441, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21570.8555, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20646.7324, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20537.5586, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20609.2578, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20168.6504, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20183.7617, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21104.4941, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21474.7969, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21322.9199, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21733.2480, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20674.8691, device='cuda:0', grad_fn=<AddBackward0>)
tensor(21726.9375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20735.9023, device='cuda:0', grad_fn=<AddBackward0>)
tensor(20616.7090, device='cuda:0', grad_fn=<AddBackward0>)


100%|██████████| 20/20 [06:42<00:00, 20.13s/it]

tensor(20704.5410, device='cuda:0', grad_fn=<AddBackward0>)


In [4]:
t.save(model.state_dict(), 'models/state_dicts/03_03.pth')